In [1]:
# Exercise 17: Creating a neural network.
#1. Open jupyter notebook
#2. Import pandas so we can store the data in dataframe (df)
import pandas as pd
df = pd.read_csv('./Exercise17/train_comment_small_50.csv',sep=',')
#3. Import regular expressions package 
import re
#4. Create a func to preproces the reviews by removing HTML tags, escaped quotes and normal quotes
def clean_comment(text):
    #Strip HTML tags
    text = re.sub('<[^<]+?>', ' ', text)
    #Strip escaped quotes
    text = text.replace('\\"', '')
    # text = re.sub('//"', '', text)
    #Strip quotes
    text = text.replace('"', '')
    # text = re.sub('"', '', text)
    return text  

#5. Apply this function to the reviews currently in the dataframe 
# Apply function clean_comment to column comment_text
df['clean_comment'] = df['comment_text'].apply(clean_comment)

#  https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
# View data with: df ; df[0:5]; df.head() ; df.tail(3) ; df.index ; df.columns ; df.to_numpy() etc.
#  Slicing rows:  df.iloc[1:3, :]
# pieces = [df[0:1], df[2:4]]
# pd.concat(pieces)

#6. Import train_test_split from scikit-learn to divide this data into Training set and a Validation set (Train on 0.8, Test on 0.2, X: text comment, y: label)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'],df['toxic'], test_size=0.2)

#7. Import nltk and stopwords from nltk lib
import nltk
nltk.download('stopwords')
#8. ML or DL need numerical data as input, currently our data is text form. Thus, we ll use CountVectorizer to convert the words present in reviews into word count vectors 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

vectorizer = CountVectorizer(binary = True, stop_words=stopwords.words('english'),
                            lowercase=True, min_df=3, max_df=0.9, max_features = 5000)

X_train_onehot = vectorizer.fit_transform(X_train)
# print("vectorizer.get_feature_names() is ","\n",len(vectorizer.get_feature_names()))

#9. Now create 2 Layers Neural Net. (2 layers not include the input layer => 
# 2 layers Neral Network[1 input layer, 1 hidden layer, 1 output layer])
#10. Import the model and layers from Keras

from keras.models import Sequential
from keras.layers import Dense

#11. Initiate the neural network 
NN = Sequential()

#12. Add the hidden layer. Specify number of nodes the layer will have the activa
NN.add(Dense(units= 500, activation='relu', input_dim=len(vectorizer.get_feature_names())))
#13. Add the output layer. 
NN.add(Dense(units=1,activation='sigmoid'))

#14. Compile the NN, decide which loss function, optimazation algorithm and performance metric we want to use. 
NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#15. Summarize our model
NN.summary()

#16. Train the model (train on the first 80, validate with the last 20)
NN.fit(X_train_onehot[:-20], y_train[:-20], epochs = 15, batch_size = 128, verbose = 1, validation_data = (X_train_onehot[-20:],y_train[-20:]))
#DONE Training now READY for Testing

#17. Transform Input Validation data (test) into word count vector, and evaluate the NN.
scores = NN.evaluate(vectorizer.transform(X_test), y_test,verbose=1)
print("Accuracy:",scores[1])

# save your model 
NN.save('NN.hd5')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               28500     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 501       
Total params: 29,001
Trainable params: 29,001
Non-trainable params: 0
_________________________________________________________________
Train on 20 samples, validate on 20 samples
Epoch 1/15
20/20 [==============================] - 0s 5ms/step - loss: 0.6932 - accuracy: 0.4000 - val_loss: 0.6901 - val_accuracy: 0.5500
Epoch 2/15
20/20 [==============================] - 0s 150us/step - loss: 0.6407 - accuracy: 0.8000 - val_loss: 0.6507 - val_accuracy: 0.6500
